In [5]:
BASE_DIR = '/media/suhail/Data/experiments/reexec/res/'

In [39]:
NB_NAME = 'nb_331056.ipynb'

In [40]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from lineage import similarity, graphs

import dataset as ds
import clustering

import nppo

In [41]:
df_dict = ds.build_df_dict_dir(BASE_DIR+NB_NAME+'/artifacts/')

In [42]:
# Build Schema, Df and index lists
dfs = []
schemas = []
for name, df in df_dict.items():
    dfs.append(name)
    schemas.append(frozenset(df))

indices = np.arange(len(schemas))

In [43]:
# Distance Metric
def set_jaccard_distance(set1,set2):
    intersect = set1.intersection(set2)
    union = set1.union(set2)
    return 1-(len(intersect)/len(union))

#set_jaccard_distance(schemas[0],schemas[0])

In [56]:
from sklearn.cluster import DBSCAN
def jacc_metric(x, y):
    i, j = int(x[0]), int(y[0])     # extract indices
    return set_jaccard_distance(schemas[i], schemas[j])

X = np.arange(len(schemas)).reshape(-1, 1)
DBSCAN(metric=jacc_metric, eps=0.1, min_samples=2).fit_predict(X)

array([ 0,  1,  1,  0,  1,  1,  1, -1,  1,  1])

In [61]:
from collections import defaultdict

def dbscan(X):
    return DBSCAN(metric=jacc_metric, eps=0.1, min_samples=2).fit_predict(X)


def generate_cluster_dict(df_dict,cluster_function):
    # Build Schema, Df and index lists
    dfs = []
    schemas = []
    for name, df in df_dict.items():
        dfs.append(name)
        schemas.append(frozenset(df))

    X = np.arange(len(schemas)).reshape(-1, 1)
    
    clusters = cluster_function(X)
    
    cluster_assigns = defaultdict(list)
    
    for i,df in enumerate(dfs):
        cluster_assigns[clusters[i]].append(df)
        
    return cluster_assigns

def new_clusters_for_noise(clusters): 
    if -1 in clusters:
        noisy_points = clusters[-1]:
        start_index = len(clusters)
        for i, point in noisy_points:
            clusters[start_index+i] = point

        del(clusters[-1])
    
    return clusters

In [62]:
generate_cluster_dict(df_dict, dbscan)

defaultdict(list,
            {0: ['typeLoc__1.csv', 'locationRecode.csv'],
             1: ['typeLoc.csv',
              'areaCrime.csv',
              'areaCrime__1.csv',
              'crimesByMonth.csv',
              'typeLoc__2.csv',
              'crimes.csv',
              'typeLoc__3.csv'],
             -1: ['typeOutcome.csv']})

In [63]:
import clustering

clustering.write_clusters_to_file(generate_cluster_dict(df_dict, dbscan), 'test.csv')